In [20]:
import os
import pandas as pd
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine, func, inspect
from sqlalchemy.orm import sessionmaker
import csv
from sqlalchemy.orm import Session

In [2]:
# Get raw dataset '2022-2023 NBA Player Stats - Regular.csv'

player_stats = os.path.join("Resources", "2022-2023 NBA Player Stats - Regular.csv")
player_stats_df = pd.read_csv(player_stats, encoding="ISO-8859-1")
player_stats_df.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Precious Achiuwa,C,23,TOR,33,9,23.0,4.0,8.2,...,0.697,2.1,4.3,6.4,1.1,0.7,0.7,1.2,2.2,10.4
1,2,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,...,0.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6
2,3,Bam Adebayo,C,25,MIA,52,52,35.3,8.6,15.7,...,0.806,2.7,7.3,10.1,3.3,1.2,0.8,2.6,2.8,21.6
3,4,Ochai Agbaji,SG,22,UTA,35,1,14.0,1.5,3.2,...,0.625,0.6,1.0,1.6,0.5,0.1,0.1,0.3,1.4,4.1
4,5,Santi Aldama,PF,22,MEM,52,18,22.0,3.4,7.0,...,0.730,1.0,3.7,4.7,1.2,0.7,0.7,0.7,1.9,9.5


In [3]:
player_stats_df.columns

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [4]:
player_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553 entries, 0 to 552
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rk      553 non-null    int64  
 1   Player  553 non-null    object 
 2   Pos     553 non-null    object 
 3   Age     553 non-null    int64  
 4   Tm      553 non-null    object 
 5   G       553 non-null    int64  
 6   GS      553 non-null    int64  
 7   MP      553 non-null    float64
 8   FG      553 non-null    float64
 9   FGA     553 non-null    float64
 10  FG%     553 non-null    float64
 11  3P      553 non-null    float64
 12  3PA     553 non-null    float64
 13  3P%     553 non-null    float64
 14  2P      553 non-null    float64
 15  2PA     553 non-null    float64
 16  2P%     553 non-null    float64
 17  eFG%    553 non-null    float64
 18  FT      553 non-null    float64
 19  FTA     553 non-null    float64
 20  FT%     553 non-null    float64
 21  ORB     553 non-null    float64
 22  DR

In [5]:
player_stats_na = player_stats_df.dropna(how = "any")
player_stats_na.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Precious Achiuwa,C,23,TOR,33,9,23.0,4.0,8.2,...,0.697,2.1,4.3,6.4,1.1,0.7,0.7,1.2,2.2,10.4
1,2,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,...,0.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6
2,3,Bam Adebayo,C,25,MIA,52,52,35.3,8.6,15.7,...,0.806,2.7,7.3,10.1,3.3,1.2,0.8,2.6,2.8,21.6
3,4,Ochai Agbaji,SG,22,UTA,35,1,14.0,1.5,3.2,...,0.625,0.6,1.0,1.6,0.5,0.1,0.1,0.3,1.4,4.1
4,5,Santi Aldama,PF,22,MEM,52,18,22.0,3.4,7.0,...,0.730,1.0,3.7,4.7,1.2,0.7,0.7,0.7,1.9,9.5


In [6]:
player_stats_cleaned = player_stats_na.drop_duplicates(subset = ["Player"])
player_stats_cleaned.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Precious Achiuwa,C,23,TOR,33,9,23.0,4.0,8.2,...,0.697,2.1,4.3,6.4,1.1,0.7,0.7,1.2,2.2,10.4
1,2,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,...,0.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6
2,3,Bam Adebayo,C,25,MIA,52,52,35.3,8.6,15.7,...,0.806,2.7,7.3,10.1,3.3,1.2,0.8,2.6,2.8,21.6
3,4,Ochai Agbaji,SG,22,UTA,35,1,14.0,1.5,3.2,...,0.625,0.6,1.0,1.6,0.5,0.1,0.1,0.3,1.4,4.1
4,5,Santi Aldama,PF,22,MEM,52,18,22.0,3.4,7.0,...,0.730,1.0,3.7,4.7,1.2,0.7,0.7,0.7,1.9,9.5


In [7]:
player_stats_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505 entries, 0 to 552
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rk      505 non-null    int64  
 1   Player  505 non-null    object 
 2   Pos     505 non-null    object 
 3   Age     505 non-null    int64  
 4   Tm      505 non-null    object 
 5   G       505 non-null    int64  
 6   GS      505 non-null    int64  
 7   MP      505 non-null    float64
 8   FG      505 non-null    float64
 9   FGA     505 non-null    float64
 10  FG%     505 non-null    float64
 11  3P      505 non-null    float64
 12  3PA     505 non-null    float64
 13  3P%     505 non-null    float64
 14  2P      505 non-null    float64
 15  2PA     505 non-null    float64
 16  2P%     505 non-null    float64
 17  eFG%    505 non-null    float64
 18  FT      505 non-null    float64
 19  FTA     505 non-null    float64
 20  FT%     505 non-null    float64
 21  ORB     505 non-null    float64
 22  DR

In [8]:
player_stats_cleaned = player_stats_cleaned.rename(columns={"Rk": "Rank", "Pos": "Position", "Tm": "Team", "G": "Games_played", "GS" : "Games_started", "MP" : "Minutes_played_per_game", "FG" : "Field_goals_per_game", "FGA" : "Field_goal_attempts_per_game", "FG%" : "Field_goal_percentage", "3P" : "three_point_field_goals_per_game", "3PA" : "three_point_field_goal_attempts_per_game", "3P%" : "three_point_field_goal_percentage", "2P" : "two_point_field_goals_per_game", "2PA" : "two_point_field_goal_attempts_per_game", "2P%" : "two_point_field_goal_percentage", "eFG%" : "Effective_field_goal_percentage", "FT" : "Free_throws_per_game", "FTA" : "Free_throw_attempts_per_game", "FT%" : "Free_throw_percentage", "ORB" : "Offensive_rebounds_per_game", "DRB" : "Defensive_rebounds_per_game", "TRB" : "Total_rebounds_per_game", "AST" : "Assists_per_game", "STL" : "Steals_per_game", "BLK" : "Blocks_per_game", "TOV" : "Turnovers_per_game", "PF" : "Personal_fouls_per_game", "PTS" : "Points_per_game"})
player_stats_cleaned.head()

,Rank,Player,Position,Age,Team,Games_played,Games_started,Minutes_played_per_game,Field_goals_per_game,Field_goal_attempts_per_game,...,Free_throw_percentage,Offensive_rebounds_per_game,Defensive_rebounds_per_game,Total_rebounds_per_game,Assists_per_game,Steals_per_game,Blocks_per_game,Turnovers_per_game,Personal_fouls_per_game,Points_per_game
0,1,Precious Achiuwa,C,23,TOR,33,9,23.0,4.0,8.2,...,0.697,2.1,4.3,6.4,1.1,0.7,0.7,1.2,2.2,10.4
1,2,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,...,0.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6
2,3,Bam Adebayo,C,25,MIA,52,52,35.3,8.6,15.7,...,0.806,2.7,7.3,10.1,3.3,1.2,0.8,2.6,2.8,21.6
3,4,Ochai Agbaji,SG,22,UTA,35,1,14.0,1.5,3.2,...,0.625,0.6,1.0,1.6,0.5,0.1,0.1,0.3,1.4,4.1
4,5,Santi Aldama,PF,22,MEM,52,18,22.0,3.4,7.0,...,0.730,1.0,3.7,4.7,1.2,0.7,0.7,0.7,1.9,9.5


In [9]:
player_stats_cleaned_df = player_stats_cleaned.reset_index(drop = True)
player_stats_cleaned_df.head()

,Rank,Player,Position,Age,Team,Games_played,Games_started,Minutes_played_per_game,Field_goals_per_game,Field_goal_attempts_per_game,...,Free_throw_percentage,Offensive_rebounds_per_game,Defensive_rebounds_per_game,Total_rebounds_per_game,Assists_per_game,Steals_per_game,Blocks_per_game,Turnovers_per_game,Personal_fouls_per_game,Points_per_game
0,1,Precious Achiuwa,C,23,TOR,33,9,23.0,4.0,8.2,...,0.697,2.1,4.3,6.4,1.1,0.7,0.7,1.2,2.2,10.4
1,2,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,...,0.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6
2,3,Bam Adebayo,C,25,MIA,52,52,35.3,8.6,15.7,...,0.806,2.7,7.3,10.1,3.3,1.2,0.8,2.6,2.8,21.6
3,4,Ochai Agbaji,SG,22,UTA,35,1,14.0,1.5,3.2,...,0.625,0.6,1.0,1.6,0.5,0.1,0.1,0.3,1.4,4.1
4,5,Santi Aldama,PF,22,MEM,52,18,22.0,3.4,7.0,...,0.730,1.0,3.7,4.7,1.2,0.7,0.7,0.7,1.9,9.5


In [10]:
player_stats_cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 30 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Rank                                      505 non-null    int64  
 1   Player                                    505 non-null    object 
 2   Position                                  505 non-null    object 
 3   Age                                       505 non-null    int64  
 4   Team                                      505 non-null    object 
 5   Games_played                              505 non-null    int64  
 6   Games_started                             505 non-null    int64  
 7   Minutes_played_per_game                   505 non-null    float64
 8   Field_goals_per_game                      505 non-null    float64
 9   Field_goal_attempts_per_game              505 non-null    float64
 10  Field_goal_percentage                 

In [11]:
player_stats_cleaned_df.to_csv("player_stats.csv", index=False)

In [12]:
Base = declarative_base()

In [13]:
class player_stats(Base):
    __tablename__ = 'player_stats'
    __table_args__ = {'sqlite_autoincrement': True}
    id = Column(Integer, primary_key=True)
    Rank = Column(Integer)                                       
    Player = Column(String)                                    
    Position = Column(String)                                  
    Age = Column (Integer)                                  
    Team = Column(String)                                     
    Games_played = Column (Integer)                             
    Games_started = Column (Integer)                                
    Minutes_played_per_game = Column (Float)                 
    Field_goals_per_game = Column (Float)                       
    Field_goal_attempts_per_game = Column (Float)          
    Field_goal_percentage = Column (Float)                      
    three_point_field_goals_per_game = Column (Float)               
    three_point_field_goal_attempts_per_game = Column (Float)       
    three_point_field_goal_percentage = Column (Float)              
    two_point_field_goals_per_game = Column (Float)               
    two_point_field_goal_attempts_per_game = Column (Float)       
    two_point_field_goal_percentage = Column (Float)              
    Effective_field_goal_percentage = Column (Float)            
    Free_throws_per_game = Column (Float)                       
    Free_throw_attempts_per_game = Column (Float)               
    Free_throw_percentage = Column (Float)                      
    Offensive_rebounds_per_game = Column (Float)                
    Defensive_rebounds_per_game = Column (Float)              
    Total_rebounds_per_game = Column (Float)                    
    Assists_per_game = Column (Float)                           
    Steals_per_game = Column (Float)                            
    Blocks_per_game = Column (Float)                           
    Turnovers_per_game = Column (Float)                         
    Personal_fouls_per_game = Column (Float)                    
    Points_per_game = Column (Float)                            

In [14]:
engine = create_engine('sqlite:///player_stats.sqlite')
Base.metadata.create_all(engine)
file_name = 'output_data/player_stats.csv'
df = pd.read_csv(file_name, encoding="ISO-8859-1")
df.to_sql(con=engine, index_label='id', name=player_stats.__tablename__, if_exists='replace')

In [16]:
Base = automap_base()
Base.prepare(engine, reflect = True)

In [17]:
Base.classes.keys()

['player_stats']

In [18]:
player_stats = Base.classes.player_stats

In [21]:
session = Session(engine)

In [22]:
inspector = inspect(engine)

In [24]:
columns = inspector.get_columns('player_stats')
for column in columns:
    print(column["name"], column["type"])

id BIGINT
Rank BIGINT
Player TEXT
Position TEXT
Age BIGINT
Team TEXT
Games_played BIGINT
Games_started BIGINT
Minutes_played_per_game FLOAT
Field_goals_per_game FLOAT
Field_goal_attempts_per_game FLOAT
Field_goal_percentage FLOAT
three_point_field_goals_per_game FLOAT
three_point_field_goal_attempts_per_game FLOAT
three_point_field_goal_percentage FLOAT
two_point_field_goals_per_game FLOAT
two_point_field_goal_attempts_per_game FLOAT
two_point_field_goal_percentage FLOAT
Effective_field_goal_percentage FLOAT
Free_throws_per_game FLOAT
Free_throw_attempts_per_game FLOAT
Free_throw_percentage FLOAT
Offensive_rebounds_per_game FLOAT
Defensive_rebounds_per_game FLOAT
Total_rebounds_per_game FLOAT
Assists_per_game FLOAT
Steals_per_game FLOAT
Blocks_per_game FLOAT
Turnovers_per_game FLOAT
Personal_fouls_per_game FLOAT
Points_per_game FLOAT
